In [109]:
import pandas as pd

df_emails = pd.read_excel("Bases de Dados/Emails.xlsx")


for i in df_emails.index:
    df_emails.loc[i, "E-mail"] = f"ric.accorsi+{i + 1}@gmail.com"
    
df_emails.to_excel("Bases de Dados/Emails.xlsx", index=False)
    
display(df_emails)

,Loja,Gerente,E-mail
0,Iguatemi Esplanada,Helena,ric.accorsi+1@gmail.com
1,Shopping Midway Mall,Alice,ric.accorsi+2@gmail.com
2,Norte Shopping,Laura,ric.accorsi+3@gmail.com
3,Shopping Iguatemi Fortaleza,Manuela,ric.accorsi+4@gmail.com
4,Shopping União de Osasco,Valentina,ric.accorsi+5@gmail.com
5,Shopping Center Interlagos,Sophia,ric.accorsi+6@gmail.com
6,Rio Mar Recife,Isabella,ric.accorsi+7@gmail.com
7,Salvador Shopping,Heloisa,ric.accorsi+8@gmail.com
8,Rio Mar Shopping Fortaleza,Luiza,ric.accorsi+9@gmail.com
9,Shopping Center Leste Aricanduva,Julia,ric.accorsi+10@gmail.com


                                Loja      Gerente                    E-mail
0                 Iguatemi Esplanada       Helena   ric.accorsi+1@gmail.com
1               Shopping Midway Mall        Alice   ric.accorsi+2@gmail.com
2                     Norte Shopping        Laura   ric.accorsi+3@gmail.com
3        Shopping Iguatemi Fortaleza      Manuela   ric.accorsi+4@gmail.com
4           Shopping União de Osasco    Valentina   ric.accorsi+5@gmail.com
5         Shopping Center Interlagos       Sophia   ric.accorsi+6@gmail.com
6                     Rio Mar Recife     Isabella   ric.accorsi+7@gmail.com
7                  Salvador Shopping      Heloisa   ric.accorsi+8@gmail.com
8         Rio Mar Shopping Fortaleza        Luiza   ric.accorsi+9@gmail.com
9   Shopping Center Leste Aricanduva        Julia  ric.accorsi+10@gmail.com
10                 Ribeirão Shopping       Lorena  ric.accorsi+11@gmail.com
11                  Shopping Morumbi        Livia  ric.accorsi+12@gmail.com
12         P

In [91]:
import pandas as pd
import win32com.client as win32 # controla todo o computador, vai permitir acessar programas da microsoft instalados no pc

from datetime import datetime
from pathlib import Path


# metas
FAT_ANO = 1650000
FAT_DIA = 1000
DIV_ANO =120
DIV_DIA = 4
TM_ANO = 500
TM_DIA = 500


def main():
    
    # pegar df's
    df_vendas = pd.read_excel("Bases de Dados/Vendas.xlsx")
    df_emails = pd.read_excel("Bases de Dados/Emails.xlsx")
    df_lojas = pd.read_csv("Bases de Dados/Lojas.csv", encoding="latin1", sep=";")
    
    
    # juntar df das lojas com a de vendas
    df_vendas = df_vendas.merge(df_lojas, on="ID Loja")
    
    # excluir coluna ID Loja
    df_vendas = df_vendas.drop("ID Loja", axis=1)
    
    # verificar se é necessário algum tratamento de dados prévio
    '''print(df_vendas.info())'''
    
    # pegar nome das lojas
    lojas = list(df_vendas["Loja"].unique())
    
    # centralizar dados anuais
    indicadores = {}
    ind_temp = {}
    
    
    # loop para executar em todas as lojas
    for loja in lojas:
        
        # pegar planilha apenas com info de cada loja
        df_loja = df_vendas.loc[df_vendas["Loja"] == loja, :]
        
        
        # cálculo de indicadores (anual)
    
        # faturamento anual
        alcancado_fat_ano = float(df_loja["Valor Final"].sum()) # fat anual
    
        # diversidade de produtos anual   
        df_div_ano = df_loja["Produto"].unique()
        alcancado_div_ano = len(df_div_ano)
    
        # ticket médio
        df_tm_ano = df_loja[["Valor Final", "Código Venda"]].groupby("Código Venda").sum()
        alcancado_tm_ano = df_tm_ano["Valor Final"].mean()
    
        # centralizar informações
        ind_temp["Anual"] = {
            "Faturamento": {
                "Meta": FAT_ANO,
                "Alcançado": alcancado_fat_ano,
                "Resultado": "green" if alcancado_fat_ano >= FAT_ANO else "red"
            },
            "Diversidade de Produtos": {
                "Meta": DIV_ANO,
                "Alcançado": alcancado_div_ano,
                "Resultado": "green" if alcancado_div_ano >= DIV_ANO else "red"
            },
                "Tícket Médio": {
                    "Meta": TM_ANO,
                    "Alcançado": alcancado_tm_ano,
                    "Resultado": "green" if alcancado_tm_ano >= TM_ANO else "red"
            }
        }
        
        
        # cálculo de indicadores (diário)
        
        # pegar último lançamento de data
        data_atual_loja = df_loja["Data"].max()
        data_atual_total = df_vendas["Data"].max()
        
        # pegar df apenas com loja individual e última data
        df_ind_dia = df_loja.loc[df_loja["Data"] == data_atual, :]
        
        # faturamento diário
        alcancado_fat_dia = df_ind_dia["Valor Final"].sum()
        
        # diversidade de produtos diária
        df_div_dia = df_ind_dia["Produto"].unique()
        alcancado_div_dia = len(df_div_dia)
        
        # tícket médio diário
        alcancado_tm_dia = df_ind_dia["Valor Final"].mean()
        
        if data_atual_loja != data_atual_total:
            alcancado_fat_dia = 0
            alcancado_div_dia = 0
            alcancado_tm_dia = 0
        
        # centralizar informações
        ind_temp["Diário"] = {
            "Data": data_atual.strftime("%d/%m/%Y"),
            "Indicadores": {
                "Faturamento": {
                    "Meta": FAT_DIA,
                    "Alcançado": alcancado_fat_dia,
                    "Resultado": "green" if alcancado_fat_dia >= FAT_DIA else "red"
                },
                "Diversidade de Produtos": {
                    "Meta": DIV_DIA,
                    "Alcançado": alcancado_div_dia,
                    "Resultado": "green" if alcancado_div_dia >= DIV_DIA else "red"
                },
                "Tícket Médio": {
                    "Meta": TM_DIA,
                    "Alcançado": alcancado_tm_dia,
                    "Resultado": "green" if alcancado_tm_dia >= TM_DIA else "red"
                }
            }
        }
        
        indicadores[loja] = ind_temp.copy()
        ind_temp.clear()
    
    
        # salvar df's de cada loja
        df_loja.to_excel(f"Lojas/{loja}.xlsx", index=False)
        
        # enviar e-mail para cada gerente
        
        info_emails = df_emails.loc[df_emails["Loja"] == loja, :] # filtrar df de emails
        
        outlook = win32.Dispatch("outlook.application") # abrir outlook
        
        mail = outlook.CreateItem(0) # criar email
        
        mail.To = f"{info_emails['E-mail']}" # destinatário
        
        mail.Subject = "Relatório de Indicadores" # assunto do email
        
        mail.HTMLBody = f'''
        <p>Bom dia, {info_emails["Gerente"].values[0]}!</p>
        
        <p>Segue o informativo matinal acerca dos indicadores anuais e diários (referente a {data_atual_total}) solicitados:</p>
        
        <table>
        
        <tr style="text-align: center">
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Situação Dia</th>
        </tr>
        
        <tr style="text-align: center">
        <td>Faturamento</th>
        <td>R$ {indicadores[loja]["Diário"]["Indicadores"]["Faturamento"]["Alcançado"]:,.2f}</th>
        <td>R$ {FAT_DIA:,.2f}</th>
        <td><font color="{indicadores[loja]["Diário"]["Indicadores"]["Faturamento"]["Resultado"]}">◙</font></th>
        </tr>
        
        <tr style="text-align: center">
        <td>Diversidade</th>
        <td>{indicadores[loja]["Diário"]["Indicadores"]["Diversidade de Produtos"]["Alcançado"]}</th>
        <td>{DIV_DIA}</th>
        <td><font color="{indicadores[loja]["Diário"]["Indicadores"]["Diversidade de Produtos"]["Resultado"]}">◙</font></th>
        </tr>
        
        <tr style="text-align: center">
        <td>Tícket Médio</th>
        <td>R$ {indicadores[loja]["Diário"]["Indicadores"]["Tícket Médio"]["Alcançado"]:,.2f}</th>
        <td>R$ {TM_DIA:,.2f}</th>
        <td><font color="{indicadores[loja]["Diário"]["Indicadores"]["Tícket Médio"]["Resultado"]}">◙</font></th>
        </tr>
        
        </table>
        
        <table>
        
        <tr style="text-align: center">
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Situação Ano</th>
        </tr>
        
        <tr style="text-align: center">
        <td>Faturamento</th>
        <td>R$ {indicadores[loja]["Anual"]["Faturamento"]["Alcançado"]:,.2f}</th>
        <td>R$ {FAT_ANO:,.2f}</th>
        <td><font color="{indicadores[loja]["Anual"]["Faturamento"]["Resultado"]}">◙</font></th>
        </tr>
        
        <tr style="text-align: center">
        <td>Diversidade</th>
        <td>{indicadores[loja]["Anual"]["Diversidade de Produtos"]["Alcançado"]}</th>
        <td>{DIV_ANO}</th>
        <td><font color="{indicadores[loja]["Anual"]["Diversidade de Produtos"]["Resultado"]}">◙</font></th>
        </tr>
        
        <tr style="text-align: center">
        <td>Tícket Médio</th>
        <td>R$ {indicadores[loja]["Anual"]["Tícket Médio"]["Alcançado"]:,.2f}</th>
        <td>R$ {TM_ANO:,.2f}</th>
        <td><font color="{indicadores[loja]["Anual"]["Tícket Médio"]["Resultado"]}">◙</font></th>
        </tr>
        
        </table>
        
        <p>Me coloco à disposição caso hajam quaisquer dúvidas. Segue em anexo a planilha completa para mais informações.</p>
        
        <p>Att,</p>
        
        <p>Ricardo</p>
        '''
        
        # pegar e enviar anexo
        caminho = Path.cwd()
        anexo = caminho / f"Lojas/{loja}.xlsx"
        
        mail.Attachments.Add(str(anexo))
        mail.Send()

    
    # pegar rankings
    ranking_ano = df_vendas[["Loja", "Valor Final"]].groupby("Loja").sum()
    ranking_ano = ranking_ano.sort_values("Valor Final", ascending=False)
    
    ranking_dia = df_vendas[["Data", "Valor Final", "Loja"]].loc[df_vendas["Data"] == df_vendas["Data"].max(), :]
    ranking_dia = ranking_dia[["Loja", "Valor Final"]].groupby("Loja").sum()
    ranking_dia = ranking_dia.sort_values("Valor Final", ascending=False)
    
    # exportar para excel
    ranking_ano.to_excel(f"Lojas/Backup/Ranking_Anual.xlsx", index=False)
    ranking_dia.to_excel(f"Lojas/Backup/Ranking_diario.xlsx", index=False)
    
    # enviar e-mail à diretoria
    outlook = win32.Dispatch("outlook.application") # abrir outlook
        
    mail = outlook.CreateItem(0) # criar email
        
    mail.To = f"{df_emails.loc[df_emails['Gerente'] == 'Diretoria', 'E-mail']}" # destinatário
        
    mail.Subject = "Ranking das Lojas" # assunto do email
    
    melhor_dia = ranking_dia.index[0]
    fat_melhor_dia = float(ranking_dia["Valor Final"][0])
    
    pior_dia = ranking_dia.index[-1]
    fat_pior_dia = float(ranking_dia["Valor Final"][-1])
                         
    melhor_ano = ranking_ano.index[0]
    fat_melhor_ano = float(ranking_ano["Valor Final"][0])
                         
    pior_ano = ranking_ano.index[-1]
    fat_pior_ano = float(ranking_ano["Valor Final"][-1])
                         
    texto = f"""
    Bom dia,
    
    RANKING DIÁRIO:
    
    Melhor loja: {melhor_dia} (R$ {fat_melhor_dia:,.2f})
    Pior loja: {pior_dia} (R$ {fat_pior_dia:,.2f})
    
    RANKING ANUAL:
    
    Melhor loja: {melhor_ano} (R$ {fat_melhor_ano:,.2f})
    Pior loja: {pior_ano} (R$ {fat_pior_ano:,.2f})
    
    Segue, em anexo, o ranking completo.
    
    Att,
    
    Ricardo
    """
    
    mail.Body = texto
    
    # pegar e enviar anexo
    caminho = Path.cwd()
    
    pasta = caminho / "Lojas/Backup"
    
    for arquivo in pasta.iterdir():
        anexo = arquivo
        mail.Attachments.Add(str(anexo)) # precisa passar uma string
    
    mail.Send()

    print("Programa concluído com sucesso")


if __name__ == "__main__":
    main()

Programa concluído com sucesso
